# 寒武纪RSI底背离分析

本Notebook将分析寒武纪（688256.SH）的历史RSI底背离信号，并可视化相关数据。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tushare as ts
import os
from dotenv import load_dotenv
from datetime import datetime, timedelta

# 导入项目模块
from signals.calculator import calculate_rsi, detect_rsi_divergence

# 加载环境变量
load_dotenv()

# 设置Tushare API令牌
ts.set_token(os.getenv('TUSHARE_TOKEN'))
pro = ts.pro_api()

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

## 1. 获取寒武纪历史数据

首先，我们获取寒武纪近两年的历史股价数据。

In [ ]:
# 寒武纪股票代码
stock_code = "688256.SH"

# 设置分析时间范围（最近2年）
end_date = datetime.now().strftime('%Y%m%d')
start_date = (datetime.now() - timedelta(days=730)).strftime('%Y%m%d')

print(f"正在获取 {stock_code} 从 {start_date} 到 {end_date} 的数据...")

# 获取股票数据
df = ts.pro_bar(ts_code=stock_code, adj='qfq', start_date=start_date, end_date=end_date)

if df is not None and not df.empty:
    # 设置日期为索引
    df['trade_date'] = pd.to_datetime(df['trade_date'])
    df.set_index('trade_date', inplace=True)
    df.sort_index(inplace=True)
    print(f"成功获取到 {len(df)} 天的数据")
else:
    print(f"未能获取到 {stock_code} 的数据")

## 2. 计算RSI指标

接下来，我们计算寒武纪的RSI指标。

In [ ]:
# 计算RSI
rsi = calculate_rsi(df)

# 将RSI添加到数据框中
df['rsi'] = rsi

# 显示前几行数据
df[['close', 'rsi']].head()

## 3. 检测RSI底背离信号

现在我们检测寒武纪的RSI底背离信号。

In [ ]:
# 检测RSI背离
divergence_signals = detect_rsi_divergence(df, rsi, period=30)

# 提取牛市背离信号（底背离）
bullish_divergence = divergence_signals['rsi_bullish_divergence']

# 查找出现底背离的日期
divergence_dates = bullish_divergence[bullish_divergence].index.tolist()

if divergence_dates:
    print(f"发现 {len(divergence_dates)} 次RSI底背离信号:")
    for date in divergence_dates:
        # 获取该日期的价格和RSI值
        price = df.loc[date, 'close']
        rsi_value = df.loc[date, 'rsi']
        print(f"  - 日期: {date.strftime('%Y-%m-%d')}, 收盘价: {price:.2f}, RSI: {rsi_value:.2f}")
else:
    print("未发现RSI底背离信号")

## 4. 可视化RSI底背离信号

最后，我们将寒武纪的股价和RSI指标可视化，并标记出底背离信号点。

In [ ]:
# 创建可视化图表
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10), sharex=True)

# 绘制股价
ax1.plot(df.index, df['close'], label='收盘价', color='blue')
ax1.set_title(f'{stock_code} 股价走势')
ax1.set_ylabel('价格')
ax1.legend()
ax1.grid(True)

# 标记底背离信号点
if divergence_dates:
    for date in divergence_dates:
        ax1.scatter(date, df.loc[date, 'close'], color='red', s=100, marker='^', label='底背离信号' if date == divergence_dates[0] else "")

# 绘制RSI
ax2.plot(df.index, df['rsi'], label='RSI', color='orange')
ax2.axhline(y=70, color='r', linestyle='--', alpha=0.7, label='超买线')
ax2.axhline(y=30, color='g', linestyle='--', alpha=0.7, label='超卖线')
ax2.axhline(y=50, color='k', linestyle='-', alpha=0.5)
ax2.set_title(f'{stock_code} RSI指标')
ax2.set_ylabel('RSI')
ax2.set_xlabel('日期')
ax2.legend()
ax2.grid(True)

# 标记底背离信号点
if divergence_dates:
    for date in divergence_dates:
        ax2.scatter(date, df.loc[date, 'rsi'], color='red', s=100, marker='^', label='底背离信号' if date == divergence_dates[0] else "")

plt.tight_layout()
plt.show()

## 5. 保存结果

将分析结果保存到CSV文件中。

In [ ]:
# 保存结果到CSV文件
if divergence_dates:
    result_df = pd.DataFrame({
        'stock_code': [stock_code] * len(divergence_dates),
        'date': [d.strftime('%Y-%m-%d') for d in divergence_dates],
        'close_price': [df.loc[d, 'close'] for d in divergence_dates],
        'rsi_value': [df.loc[d, 'rsi'] for d in divergence_dates]
    })
    
    filename = f"{stock_code}_rsi_bullish_divergence_analysis.csv"
    result_df.to_csv(filename, index=False)
    print(f"结果已保存到 {filename}")
    
    # 显示结果
    result_df